# Random Forest Classification of Lunar GRS Data

**By Tom Sander**

---

## Overview

This notebook demonstrates how to apply **supervised machine learning** (specifically Random Forest classification) to **Lunar Prospector Gamma-Ray Spectrometer (GRS)** data. Building on the unsupervised clustering approach from the previous workshop, we will now train a model that can predict lunar terrane types based on geochemical signatures.

### Learning Objectives

By the end of this tutorial, you will:

1. Understand the difference between **supervised** and **unsupervised** learning
2. Create labeled training data for known lunar terranes
3. Train a **Random Forest classifier** on geochemical data
4. Evaluate model performance using confusion matrices and classification reports
5. Interpret **feature importance** to understand which elements drive classification
6. Apply the trained model to predict terrane types across the entire Moon

### Why Random Forest?

Random Forest is particularly well-suited for geochemical data because:
- **Handles non-linear relationships** between elements (e.g., Fe-Ti correlations in mare basalts)
- **Robust to outliers** (important for heterogeneous lunar surfaces)
- **Provides feature importance** (reveals which elements are most diagnostic)
- **No need for extensive data normalization** (unlike K-Means or neural networks)
- **Works well with small-to-medium datasets** (typical for planetary science)

---

## 1. Import Packages

We'll use the following libraries:
- **Polars**: Fast DataFrame library for data manipulation
- **Matplotlib/Seaborn**: For creating visualizations
- **Scikit-learn**: For Random Forest and model evaluation
- **NumPy**: For numerical operations

In [ ]:
# Data manipulation and numerical operations
import polars as pl
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
import seaborn as sns

# Machine learning: Random Forest and evaluation tools
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

# Set visualization style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

---

## 2. Load Lunar Prospector GRS Data

We use the same data loader function from the previous workshop to load the Lunar Prospector GRS data. This dataset contains elemental abundance measurements at 5-degree spatial resolution, covering the entire lunar surface.

**Data Source:** NASA PDS Geosciences Node - Lunar Prospector Mission (1998-1999)

In [ ]:
def load_lpgrs_tab(filepath: str) -> pl.DataFrame:
    """
    Loads a Lunar Prospector GRS .tab file into a Polars DataFrame.
    
    Handles wrapped lines, multiple-space delimiters, and scientific notation.

    Args:
        filepath (str): Path to the .tab file.

    Returns:
        pl.DataFrame: DataFrame containing the GRS data with appropriate column names.
    """
    # Read all whitespace-separated tokens (NASA records often wrap across lines)
    with open(filepath, 'r') as f:
        tokens = f.read().split()
    
    # Reshape into 61 columns (standard GRS abundance format)
    record_width = 61
    rows = [tokens[i : i + record_width] for i in range(0, len(tokens), record_width)]
    
    # Create DataFrame and cast to Float64
    df = pl.DataFrame(rows, orient="row").select([
        pl.all().cast(pl.Float64)
    ])
    
    # Define column names matching the PDS label file specification:
    # Columns 0-4: Pixel index and geographic bounds
    # Columns 5-6: Atomic mass and neutron density
    # Columns 7-12: Major oxide weight fractions (MgO, Al2O3, SiO2, CaO, TiO2, FeO)
    # Columns 13-15: Trace elements (K, Th, U in ppm)
    names = [
        "bin_index", "lat_start", "lat_end", "lon_start", "lon_end",
        "atomic_mass", "neutron_density",
        "mgo", "al2o3", "sio2", "cao", "tio2", "feo",
        "potassium", "thorium", "uranium"
    ]
    
    return df.select(df.columns[:16]).rename(
        {old: new for old, new in zip(df.columns[:16], names)}
    )

# Load the GRS data
path_to_tab = "data/lpgrs_high1_elem_abundance_5deg.tab"
df = load_lpgrs_tab(path_to_tab)

print(f"Loaded {df.height} records with {df.width} columns")
print(f"\nAvailable measurements:")
print(f"  Oxides: MgO, Al2O3, SiO2, CaO, TiO2, FeO (weight fraction)")
print(f"  Trace elements: K, Th, U (ppm)")
df.head()

---

## 3. Understanding Lunar Terranes: Creating Training Labels

Unlike unsupervised learning (K-Means), **supervised learning** requires labeled training data. We will define the three major lunar terranes based on well-established geochemical and geographic criteria:

### The Three Major Lunar Terranes

| Terrane | Abbreviation | Key Characteristics | Geographic Location |
|---------|--------------|---------------------|---------------------|
| **Procellarum KREEP Terrane** | PKT | High Th (>3 ppm), High K, High Fe | Nearside: Oceanus Procellarum, Mare Imbrium |
| **Feldspathic Highlands Terrane** | FHT | Low Th (<1.5 ppm), Low Fe (<8%), Low Ti | Global highlands, especially farside |
| **South Pole-Aitken Basin** | SPA | Intermediate Fe, variable Th, unique Mg signature | Farside south polar region |

These terranes were first defined by Jolliff et al. (2000) based on Lunar Prospector data and represent fundamentally different crustal compositions reflecting distinct geological histories.

> **Reference:** Jolliff, B. L., Gillis, J. J., Haskin, L. A., Korotev, R. L., & Wieczorek, M. A. (2000). Major lunar crustal terranes: Surface expressions and crust-mantle origins. *Journal of Geophysical Research*, 105(E2), 4197-4216.

In [ ]:
# Define the geochemical features for classification
# Using FeO and TiO2 (weight fractions) plus Th and K (ppm)
feature_cols = ['feo', 'tio2', 'thorium', 'potassium']

# Clean the data by removing any null values
df_clean = df.drop_nulls(subset=feature_cols)

# Calculate center coordinates for each bin (for geographic labeling)
df_clean = df_clean.with_columns([
    ((pl.col("lat_start") + pl.col("lat_end")) / 2).alias("lat_center"),
    ((pl.col("lon_start") + pl.col("lon_end")) / 2).alias("lon_center")
])

print(f"Clean dataset: {df_clean.height} pixels ready for analysis")
print(f"\nFeature columns: {feature_cols}")
print(f"  - feo: FeO weight fraction (g/g)")
print(f"  - tio2: TiO2 weight fraction (g/g)")
print(f"  - thorium: Th concentration (ppm)")
print(f"  - potassium: K concentration (ppm)")

### 3.1 Automated Terrane Labeling

We'll assign terrane labels based on combined **geochemical** and **geographic** criteria. This approach mimics how a lunar geologist would classify terranes using both compositional data and spatial context.

**Classification Rules:**
1. **SPA Basin**: Latitude < -30° AND Longitude between 120° and 240° (farside) AND intermediate Fe
2. **PKT**: Thorium > 3 ppm (clear KREEP signature)
3. **FHT**: Everything else (low-Th highlands)

In [ ]:
def assign_terrane_label(row: dict) -> str:
    """
    Assigns a terrane label based on geochemical and geographic criteria.
    
    This follows the terrane definitions from Jolliff et al. (2000):
    - PKT: High thorium regions (KREEP-rich)
    - SPA: South Pole-Aitken Basin (farside, high-Fe impact basin)
    - FHT: Feldspathic Highlands (low-Th, low-Fe highlands)
    
    Note: FeO is stored as weight fraction (g/g), so 0.10 = 10 wt%
          Thorium is stored in ppm
    """
    lat = row["lat_center"]
    lon = row["lon_center"]
    th = row["thorium"]       # ppm
    feo = row["feo"]          # weight fraction (g/g), multiply by 100 for wt%
    
    # Convert FeO weight fraction to wt% for easier interpretation
    fe_wt_pct = feo * 100
    
    # South Pole-Aitken Basin: farside south polar region with elevated Fe
    # SPA spans roughly 120°E to -120°E on the farside
    # and latitudes from about -20° to -60°
    is_farside_south = lat < -20 and (lon > 120 or lon < -120)
    
    if is_farside_south and fe_wt_pct > 8 and th < 3:
        return "SPA"
    # PKT: High thorium indicates KREEP-rich material (>3 ppm Th)
    elif th > 3.0:
        return "PKT"
    # FHT: Default - low thorium highlands
    else:
        return "FHT"

# Apply terrane labeling to each pixel
terrane_labels = []
for i in range(df_clean.height):
    row = df_clean.row(i, named=True)
    terrane_labels.append(assign_terrane_label(row))

# Add terrane labels to the DataFrame
df_labeled = df_clean.with_columns(
    pl.Series(name="Terrane", values=terrane_labels)
)

# Display terrane distribution
print("=" * 50)
print("TERRANE DISTRIBUTION")
print("=" * 50)
terrane_counts = df_labeled.group_by("Terrane").agg(pl.len().alias("Count")).sort("Terrane")
for row in terrane_counts.iter_rows(named=True):
    pct = row["Count"] / df_labeled.height * 100
    print(f"  {row['Terrane']}: {row['Count']} pixels ({pct:.1f}%)")
print("=" * 50)

### 3.2 Visualize the Training Labels

Before training our model, let's visualize the assigned terrane labels to verify they match known lunar geology. The PKT should appear on the nearside (around Oceanus Procellarum), the SPA Basin on the farside south polar region, and the FHT should dominate the highlands.

In [ ]:
# Create a color mapping for terranes (geologically meaningful colors)
terrane_colors = {
    "PKT": "#d62728",  # Red - volcanic/KREEP-rich
    "FHT": "#7f7f7f",  # Gray - anorthositic highlands
    "SPA": "#1f77b4"   # Blue - ancient impact basin
}

# Create figure
fig, ax = plt.subplots(figsize=(15, 8))

# Plot each terrane as colored rectangles
for terrane, color in terrane_colors.items():
    df_terrane = df_labeled.filter(pl.col("Terrane") == terrane)
    
    for i in range(df_terrane.height):
        row = df_terrane.row(i, named=True)
        width = row['lon_end'] - row['lon_start']
        height = row['lat_end'] - row['lat_start']
        rect = Rectangle((row['lon_start'], row['lat_start']), width, height,
                         facecolor=color, edgecolor='none', alpha=0.8)
        ax.add_patch(rect)

# Create legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=terrane_colors["PKT"], label='PKT (Procellarum KREEP Terrane)'),
    Patch(facecolor=terrane_colors["FHT"], label='FHT (Feldspathic Highlands Terrane)'),
    Patch(facecolor=terrane_colors["SPA"], label='SPA (South Pole-Aitken Basin)')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=10)

# Configure axes
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.set_xlabel('Longitude (degrees)', fontsize=12)
ax.set_ylabel('Latitude (degrees)', fontsize=12)
ax.set_title('Training Labels: Lunar Terrane Classification', fontsize=14)
ax.grid(True, linestyle=':', alpha=0.6)
ax.set_aspect('equal')

plt.tight_layout()
plt.show()

---

## 4. Preparing Data for Random Forest

Now we'll prepare our data for supervised learning. Unlike K-Means, Random Forest requires:

1. **Feature matrix (X)**: The geochemical measurements
2. **Target vector (y)**: The terrane labels we want to predict
3. **Train/Test Split**: To evaluate how well our model generalizes

### Why Split the Data?

We hold out 20% of the data for testing. This simulates applying our model to "new" data it hasn't seen, giving us an honest assessment of performance.

In [ ]:
# Extract features (X) and target labels (y)
X = df_labeled.select(feature_cols).to_numpy()
y = df_labeled.select("Terrane").to_numpy().flatten()

# Split data into training (80%) and testing (20%) sets
# stratify=y ensures proportional representation of each terrane in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,  # For reproducibility
    stratify=y        # Maintain class proportions
)

print("=" * 50)
print("DATA SPLIT SUMMARY")
print("=" * 50)
print(f"Total samples:    {len(X)}")
print(f"Training samples: {len(X_train)} (80%)")
print(f"Testing samples:  {len(X_test)} (20%)")
print(f"\nFeatures used: {feature_cols}")
print(f"Classes: {np.unique(y)}")

---

## 5. Training the Random Forest Classifier

### How Random Forest Works

Random Forest is an **ensemble method** that builds multiple decision trees and combines their predictions. Here's the intuition:

1. **Bootstrap Sampling**: Each tree is trained on a random subset of the data
2. **Feature Randomization**: At each split, only a random subset of features is considered
3. **Voting**: Final prediction is determined by majority vote across all trees

This approach reduces **overfitting** and provides robust predictions even with noisy data, ideal for geochemical measurements with inherent uncertainties.

### Key Hyperparameters

| Parameter | Description | Our Setting |
|-----------|-------------|-------------|
| `n_estimators` | Number of trees in the forest | 100 |
| `max_depth` | Maximum depth of each tree | None (unlimited) |
| `min_samples_split` | Minimum samples to split a node | 2 |
| `random_state` | Seed for reproducibility | 42 |

In [ ]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(
    n_estimators=100,      # Number of trees in the forest
    max_depth=None,        # Allow trees to grow fully
    min_samples_split=2,   # Minimum samples required to split
    min_samples_leaf=1,    # Minimum samples in a leaf node
    random_state=42,       # For reproducibility
    n_jobs=-1              # Use all available CPU cores
)

# Train the model on the training data
print("Training Random Forest classifier...")
rf_classifier.fit(X_train, y_train)

print("✓ Model training complete!")
print(f"  - Number of trees: {rf_classifier.n_estimators}")
print(f"  - Features per tree: {rf_classifier.n_features_in_}")

---

## 6. Model Evaluation

Now we evaluate how well our Random Forest model performs. We'll use several metrics:

1. **Accuracy**: Overall percentage of correct predictions
2. **Confusion Matrix**: Shows where the model makes mistakes
3. **Precision & Recall**: Per-class performance metrics
4. **Cross-Validation**: Robust estimate of generalization performance

In [ ]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print("=" * 60)
print("MODEL PERFORMANCE ON TEST SET")
print("=" * 60)
print(f"\nOverall Accuracy: {accuracy:.1%}")
print("\n" + "-" * 60)
print("CLASSIFICATION REPORT")
print("-" * 60)
print(classification_report(y_test, y_pred, target_names=["FHT", "PKT", "SPA"]))

### 6.1 Confusion Matrix

The confusion matrix reveals which terranes are most easily confused with each other. This has important geological implications—if the model confuses two terranes, they may have overlapping geochemical signatures.

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["FHT", "PKT", "SPA"])

# Plot confusion matrix as a heatmap
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["FHT", "PKT", "SPA"],
            yticklabels=["FHT", "PKT", "SPA"],
            ax=ax)
ax.set_xlabel('Predicted Terrane', fontsize=12)
ax.set_ylabel('True Terrane', fontsize=12)
ax.set_title('Confusion Matrix: Random Forest Classification', fontsize=14)

plt.tight_layout()
plt.show()

# Interpretation
print("\n Interpretation:")
print("   - Diagonal values = Correct predictions")
print("   - Off-diagonal = Misclassifications")
print("   - High FHT-SPA confusion may indicate transitional zones")

### 6.2 Cross-Validation

To get a more robust estimate of model performance, we use **k-fold cross-validation**. This repeatedly splits the data different ways and averages the results, reducing the impact of any particular train/test split.

In [ ]:
# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_classifier, X, y, cv=5, scoring='accuracy')

print("=" * 50)
print("5-FOLD CROSS-VALIDATION RESULTS")
print("=" * 50)
print(f"\nFold accuracies: {[f'{s:.1%}' for s in cv_scores]}")
print(f"\nMean accuracy: {cv_scores.mean():.1%} (+/- {cv_scores.std() * 2:.1%})")
print("\n -> Low variance across folds indicates stable model performance.")

---

## 7. Feature Importance Analysis

One of the most powerful aspects of Random Forest is its ability to quantify **feature importance**. This tells us which geochemical elements are most diagnostic for terrane classification.

### Why This Matters for Lunar Geology

Understanding feature importance can:
- Confirm known geochemical relationships (e.g., Th as a KREEP tracer)
- Suggest which elements to prioritize in future missions
- Reveal unexpected discriminating factors

In [ ]:
# Extract feature importances from the trained model
importances = rf_classifier.feature_importances_
feature_importance_df = pl.DataFrame({
    "Feature": feature_cols,
    "Importance": importances
}).sort("Importance", descending=True)

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(10, 5))

# Sort for plotting
features_sorted = feature_importance_df.sort("Importance")
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(feature_cols)))

ax.barh(features_sorted["Feature"].to_list(), 
        features_sorted["Importance"].to_list(),
        color=colors)
ax.set_xlabel('Feature Importance (Gini Importance)', fontsize=12)
ax.set_title('Random Forest Feature Importance for Terrane Classification', fontsize=14)
ax.set_xlim(0, max(importances) * 1.1)

# Add value labels
for i, (feat, imp) in enumerate(zip(features_sorted["Feature"].to_list(), 
                                     features_sorted["Importance"].to_list())):
    ax.text(imp + 0.01, i, f'{imp:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

# Print interpretation
print("\n" + "=" * 60)
print("FEATURE IMPORTANCE INTERPRETATION")
print("=" * 60)
for row in feature_importance_df.iter_rows(named=True):
    print(f"  {row['Feature'].upper():12} : {row['Importance']:.3f}")
print("\n Geological Insight:")
print("   - Thorium is the key discriminator (KREEP signature)")
print("   - Iron distinguishes mare basalts from highlands")
print("   - Ti helps identify high (Ti vs low-Ti mare basalts)")

---

## 8. Prediction Map: Applying the Model to the Entire Moon

Now we apply our trained Random Forest model to predict terrane types across the entire lunar surface. This creates a **supervised classification map** that can be compared with the unsupervised K-Means results from the previous workshop.

In [ ]:
# Predict terrane labels for the entire dataset
X_all = df_labeled.select(feature_cols).to_numpy()
y_pred_all = rf_classifier.predict(X_all)

# Add predictions to the DataFrame
df_predicted = df_labeled.with_columns(
    pl.Series(name="Predicted_Terrane", values=y_pred_all)
)

# Create the prediction map
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Plot 1: Original Labels (Training Data)
ax1 = axes[0]
for terrane, color in terrane_colors.items():
    df_terrane = df_labeled.filter(pl.col("Terrane") == terrane)
    for i in range(df_terrane.height):
        row = df_terrane.row(i, named=True)
        width = row['lon_end'] - row['lon_start']
        height = row['lat_end'] - row['lat_start']
        rect = Rectangle((row['lon_start'], row['lat_start']), width, height,
                         facecolor=color, edgecolor='none', alpha=0.8)
        ax1.add_patch(rect)

ax1.set_xlim(-180, 180)
ax1.set_ylim(-90, 90)
ax1.set_xlabel('Longitude (degrees)', fontsize=11)
ax1.set_ylabel('Latitude (degrees)', fontsize=11)
ax1.set_title('Original Terrane Labels (Rule-Based)', fontsize=13)
ax1.grid(True, linestyle=':', alpha=0.6)
ax1.set_aspect('equal')

# Plot 2: Random Forest Predictions
ax2 = axes[1]
for terrane, color in terrane_colors.items():
    df_terrane = df_predicted.filter(pl.col("Predicted_Terrane") == terrane)
    for i in range(df_terrane.height):
        row = df_terrane.row(i, named=True)
        width = row['lon_end'] - row['lon_start']
        height = row['lat_end'] - row['lat_start']
        rect = Rectangle((row['lon_start'], row['lat_start']), width, height,
                         facecolor=color, edgecolor='none', alpha=0.8)
        ax2.add_patch(rect)

ax2.set_xlim(-180, 180)
ax2.set_ylim(-90, 90)
ax2.set_xlabel('Longitude (degrees)', fontsize=11)
ax2.set_ylabel('Latitude (degrees)', fontsize=11)
ax2.set_title('Random Forest Predictions', fontsize=13)
ax2.grid(True, linestyle=':', alpha=0.6)
ax2.set_aspect('equal')

# Add shared legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=terrane_colors["PKT"], label='PKT'),
    Patch(facecolor=terrane_colors["FHT"], label='FHT'),
    Patch(facecolor=terrane_colors["SPA"], label='SPA')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11,
           bbox_to_anchor=(0.5, 0.1))

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
plt.show()

---

## 9. Prediction Probability Maps

Random Forest doesn't just predict a class, it also provides **class probabilities**. These probabilities indicate the model's confidence in its prediction. Low probabilities often indicate transitional zones between terranes, which are geologically interesting regions.

In [ ]:
# Get prediction probabilities for each class
proba = rf_classifier.predict_proba(X_all)
classes = rf_classifier.classes_  # Get class order

# Add probabilities to DataFrame
df_proba = df_predicted.with_columns([
    pl.Series(name=f"Prob_{cls}", values=proba[:, i]) 
    for i, cls in enumerate(classes)
])

# Create probability maps for each terrane
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (terrane, ax) in enumerate(zip(classes, axes)):
    prob_col = f"Prob_{terrane}"
    
    # Create scatter plot with probability as color
    patches = []
    colors_list = []
    
    for i in range(df_proba.height):
        row = df_proba.row(i, named=True)
        width = row['lon_end'] - row['lon_start']
        height = row['lat_end'] - row['lat_start']
        rect = Rectangle((row['lon_start'], row['lat_start']), width, height)
        patches.append(rect)
        colors_list.append(row[prob_col])
    
    collection = PatchCollection(patches, cmap='plasma', alpha=0.9)
    collection.set_array(np.array(colors_list))
    collection.set_clim(0, 1)
    ax.add_collection(collection)
    
    # Colorbar
    plt.colorbar(collection, ax=ax, label='Probability', shrink=0.8)
    
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    ax.set_xlabel('Longitude (degrees)', fontsize=10)
    ax.set_ylabel('Latitude (degrees)', fontsize=10)
    ax.set_title(f'{terrane} Probability', fontsize=12)
    ax.grid(True, linestyle=':', alpha=0.4)
    ax.set_aspect('equal')

plt.suptitle('Random Forest Classification Probability Maps', fontsize=14, y=0.875)
plt.tight_layout()
plt.show()

print("\n Interpretation:")
print("   - High probability (yellow) = High confidence")
print("   - Low probability (purple) = Low confidence / transitional zones")
print("   - Transitional zones may represent geological boundaries or mixed compositions")

---

## 10. Geochemical Signatures by Terrane

Let's examine the average geochemical composition of each predicted terrane to verify our results match expected lunar geology.

In [ ]:
# Calculate mean geochemistry for each predicted terrane
terrane_chemistry = df_predicted.group_by("Predicted_Terrane").agg([
    (pl.col("feo") * 100).mean().alias("FeO (wt%)"),      # Convert to wt%
    (pl.col("tio2") * 100).mean().alias("TiO2 (wt%)"),    # Convert to wt%
    pl.col("thorium").mean().alias("Th (ppm)"),
    pl.col("potassium").mean().alias("K (ppm)"),
    pl.len().alias("N_Pixels")
]).sort("Predicted_Terrane")

print("=" * 70)
print("GEOCHEMICAL SIGNATURES BY PREDICTED TERRANE")
print("=" * 70)
print(terrane_chemistry)
print("=" * 70)

# Create bar chart comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

elements = ["FeO (wt%)", "TiO2 (wt%)", "Th (ppm)", "K (ppm)"]
colors = [terrane_colors[t] for t in terrane_chemistry["Predicted_Terrane"].to_list()]

for ax, element in zip(axes.flatten(), elements):
    values = terrane_chemistry[element].to_list()
    terranes = terrane_chemistry["Predicted_Terrane"].to_list()
    
    bars = ax.bar(terranes, values, color=colors, edgecolor='black', linewidth=1)
    ax.set_ylabel(element, fontsize=11)
    ax.set_title(f'Average {element.split()[0]} by Terrane', fontsize=12, y=1.02)
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(values),
                f'{val:.2f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Geochemical Fingerprints of Lunar Terranes', fontsize=14, y=1.)
plt.tight_layout()
plt.show()

print("\n Expected Signatures (from Jolliff et al., 2000):")
print("   PKT: High Th (4-10+ ppm), High K, Elevated FeO (mare influence)")
print("   FHT: Low Th (<1.5 ppm), Low FeO (<8 wt%), Anorthositic highlands")
print("   SPA: Intermediate FeO (~10 wt%), Moderate Th, Ancient impact basin")

---

## 11. Comparison: Supervised vs. Unsupervised Learning

How does our Random Forest classification compare to the unsupervised K-Means clustering from the previous workshop? Let's briefly discuss the key differences:

| Aspect | K-Means (Unsupervised) | Random Forest (Supervised) |
|--------|------------------------|---------------------------|
| **Training Data** | No labels required | Requires labeled examples |
| **Interpretability** | Clusters may not match geological units | Predictions map to known terranes |
| **Flexibility** | Discovers patterns autonomously | Learns from expert knowledge |
| **Feature Importance** | Not directly available | Quantifies element importance |
| **Confidence Estimates** | Distance to centroid | Probability for each class |
| **Best Use Case** | Exploration, finding unknowns | Classification, validation |

### When to Use Each Approach

- **K-Means**: When you don't have labeled data or want to discover unexpected patterns
- **Random Forest**: When you want to classify data according to known categories and understand which features matter most

---

## 12. Bonus: Exploring Hyperparameter Tuning

The performance of Random Forest can be improved by tuning its hyperparameters. Let's explore how the number of trees affects accuracy.

In [ ]:
# Test different numbers of trees
n_estimators_range = [10, 25, 50, 100, 200, 300]
accuracies = []

print("Testing different numbers of trees...")
print("-" * 40)

for n_est in n_estimators_range:
    rf_temp = RandomForestClassifier(n_estimators=n_est, random_state=42, n_jobs=-1)
    rf_temp.fit(X_train, y_train)
    acc = rf_temp.score(X_test, y_test)
    accuracies.append(acc)
    print(f"  n_estimators={n_est:3d}: Accuracy = {acc:.3f}")

# Plot the results
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(n_estimators_range, accuracies, marker='o', linewidth=2, markersize=8)
ax.set_xlabel('Number of Trees (n_estimators)', fontsize=12)
ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Effect of Number of Trees on Model Accuracy', fontsize=14)
ax.set_ylim(min(accuracies) - 0.01, 1.01)
ax.grid(True, alpha=0.3)

# Add annotations
for x, y in zip(n_estimators_range, accuracies):
    ax.annotate(f'{y:.3f}', (x, y), textcoords="offset points", 
                xytext=(0, 10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n Observation: Accuracy typically plateaus after 50-100 trees.")
print("   More trees = more computation but diminishing returns.")

---

## 13. Conclusions & Next Steps

### What We Accomplished

- Loaded and preprocessed Lunar Prospector GRS elemental abundance data  
- Created labeled training data based on known lunar terrane definitions  
- Trained a Random Forest classifier to predict terrane types  
- Evaluated model performance using confusion matrices and cross-validation  
- Identified **thorium** as the most important feature for terrane classification  
- Generated probability maps showing classification confidence  
- Compared supervised vs. unsupervised machine learning approaches  

### Key Geological Insights

1. **Thorium is the dominant discriminator**: This confirms its role as the primary KREEP tracer and validates the Jolliff et al. (2000) terrane definitions
2. **Iron provides secondary discrimination**: Distinguishing between mare-flooded and highland regions
3. **The model successfully identifies all three major terranes** with high accuracy

### Ideas for Further Exploration

- Add more features: Samarium, Calcium, Mg/Fe ratios
- Try other classifiers: Support Vector Machines, Gradient Boosting, Neural Networks
- Increase spatial resolution using higher-resolution datasets
- Apply to other planetary bodies: Mars (GRS from Mars Odyssey), Mercury (MESSENGER), Vesta (Dawn)
- Investigate transition zones with low prediction probability
- Combine with mineralogical data (Clementine, M³) for multimodal classification

### References

- Jolliff, B. L., et al. (2000). Major lunar crustal terranes: Surface expressions and crust-mantle origins. *JGR*, 105(E2), 4197-4216.
- Prettyman, T. H., et al. (2006). Elemental composition of the lunar surface: Analysis of gamma ray spectroscopy data from Lunar Prospector. *JGR*, 111(E12).
- Breiman, L. (2001). Random forests. *Machine Learning*, 45(1), 5-32.

---

*Workshop 3 complete! You now have the tools to apply supervised machine learning to planetary geochemical data.*